In [1]:
import funcoes as fc
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico, options=options)
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [21]:
tipoExig = 5
navegador.switch_to.default_content()
#VOLTA PARA A PAGINA INICIAL DO PROCESSO
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.102820/2024-46') 
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)    
time.sleep(1)
navegador.switch_to.frame('ifrConteudoVisualizacao')
#CLICA NO ICONE DE TAG
time.sleep(0.5)
navegador.find_element(By.XPATH, "//img[contains(@src, 'svg/marcador_gerenciar.svg?18')]").click()
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
time.sleep(0.5)
if tipoExig == 1:
    #CLICA NA TAG DE INTERCORRENTE
    navegador.find_element(By.PARTIAL_LINK_TEXT, 'Processo SEI ORCN - Intercorrente').click()
else:
    #CLICA NA TAG DE PENDENCIA
    navegador.find_element(By.PARTIAL_LINK_TEXT, 'Processo SEI ORCN - Pendência').click()
    print('exigencia')

exigencia


In [3]:
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')

if fc.check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório', navegador) and not fc.check_element_exists(By.XPATH, "//img[contains(@src, 'svg/marcador_verde.svg?18')]", navegador):
    print("O processo já foi despachado!")
    
elif fc.check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório', navegador) and fc.check_element_exists(By.XPATH, "//img[contains(@src, 'svg/marcador_verde.svg?18')]", navegador):
    tipo_processo = 'Reaberto'
    print('processo reaberto')

processo reaberto


In [8]:
planilhaDrones = f"C:\\Users\\andrej.estagio\\ANATEL\\ORCN - Drones\\Lista de Drones Anatel_Corrigida.xlsx"
planilhaRadios = f"C:\\Users\\andrej.estagio\\ANATEL\\ORCN - Rádios\\Lista Radiamador.xlsx"

In [9]:
fc.analisa(navegador, '53500.104031/2024-40', 'André Jacinto Rodrigues', fc.corrige_planilha(planilhaDrones, True), fc.corrige_planilha(planilhaRadios, False))

Não foi possível encontrar a Declaração de Conformidade.
Célula encontrada: 53500.104031/2024-40


In [11]:
processo = '53500.070427/2024-86'

In [14]:
fc.retorna_pagina_inicial(navegador, processo)
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('Despacho Decisório de Cancelamento encontrado.\n')

        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()






Despacho Decisório de Cancelamento encontrado.



In [16]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
fc.retorna_pagina_inicial(navegador, processo)
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES
fc.preenche_planilhageral(processo, 'André Jacinto Rodrigues', situacao="Cancelado")

Célula encontrada: Exigência


In [8]:
#CLICA NA OPCAO DE DESPACHO DECISORIO
navegador.switch_to.frame('ifrVisualizacao')
fc.clica_noelemento(navegador, By.PARTIAL_LINK_TEXT,'Despacho Decisório')
#navegador.find_element(By.XPATH,'//*[@id="tblSeries"]/tbody/tr[16]/td/a[2]').click()
#time.sleep(2)
#SELECIONA TEXTO PADRAO

In [9]:
fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divOptTextoPadrao"]/div')
#navegador.find_element(By.XPATH,'//*[@id="divOptTextoPadrao"]/div').click()
#time.sleep(2)
#ENVIA QUAL DESPACHO DECISORIO DEVE SER CRIADO

In [10]:
navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório - Cancelamento de Homologação')
#time.sleep(2)

In [11]:
#CLICA NA PRIMEIRA OPCAO
fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divInfraAjaxtxtTextoPadrao"]/ul/li/a')

In [12]:
#COLOCA O DESPACHO COMO PUBLICO
fc.clica_noelemento(navegador, By.XPATH,'//*[@id="divOptPublico"]/div')

In [13]:
#SALVA DESPACHO
fc.clica_noelemento(navegador, By.ID,'btnSalvar')

In [14]:

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('processo achado')
        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
        



processo achado


In [15]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.047484/2024-61')
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [16]:
fc.preenche_planilhageral('53500.047484/2024-61', 'André Jacinto Rodrigues', 'Não', 'Cancelado', '', 'Ademir Camargo')

Célula encontrada: nan
